In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import FinanceDataReader as fdr

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

kosdak = pd.read_csv('KOSDAK.csv')
kosdak.head()

FileNotFoundError: [Errno 2] No such file or directory: 'KOSDAK.csv'

In [ ]:
krx = pd.read_excel('krx/krx.xlsx')
krx.head()

In [ ]:
import pandas as pd



# 데이터프레임을 Code 컬럼을 기준으로 병합
merged_df = pd.merge(kosdak, krx, on='Code', how='inner')



# 결과 출력
merged_df.head()


In [ ]:
# 'Name_x'를 'Name'으로 이름 변경
merged_df.rename(columns={'Name_x': 'Name'}, inplace=True)

# 'Name_y' 컬럼 삭제
merged_df.drop(columns=['Name_y'], inplace=True)
merged_df.head()

In [ ]:
# 병합된 데이터프레임을 KOSDAK.csv 파일로 저장
merged_df.to_csv('KOSDAK_merged.csv', index=False)

In [ ]:
KOSDAK = pd.read_csv('KOSDAK_merged.csv')
KOSDAK.head()

In [ ]:
KOSDAK.info()

In [ ]:
KOSDAK.isnull().sum()

In [ ]:
# PER 15~20 사이
# ROE 20 

In [ ]:
KOSDAK.shape

In [ ]:
97/1746

In [ ]:
# 결측치가 있는 모든 행 제거
KOSDAK_clean = KOSDAK.dropna()

In [ ]:
KOSDAK_clean.shape

In [ ]:
# EMA(지수이동평균) 추가하기
import FinanceDataReader as fdr
import pandas as pd
import matplotlib.pyplot as plt
ecopro = fdr.DataReader('086520')
close_prices = ecopro['Close']
ema_50 = close_prices.ewm(span=50, adjust=False).mean()
ecopro['ema_50'] = ema_50
ema_12 = close_prices.ewm(span=12, adjust=False).mean()
ecopro['ema_12'] = ema_12
ema_26 = close_prices.ewm(span=26, adjust=False).mean()
ecopro['ema_26'] = ema_26
ema_200 = close_prices.ewm(span=200, adjust=False).mean()
ecopro['ema_200'] = ema_200
ecopro['MACD'] = ecopro['ema_12'] - ecopro['ema_26']
ecopro['Signal Line'] = ecopro['MACD'].ewm(span=9, adjust=False).mean()
# 결과 시각화
plt.figure(figsize=(14,7))
plt.plot(close_prices, label='Close Price', color='black')
plt.plot(ema_50, label='10-day EMA', color='blue')
plt.plot(ema_200, label='120-day EMA', color='red')
plt.plot(ecopro['MACD'], label='MACD')
plt.plot(ecopro['Signal Line'], label='Signal Line')
plt.title('ECOPRO (086520) - Close Price and EMA')
plt.legend(loc='upper left')
plt.show()
signals = [2]
for i in range(1, len(ecopro)):
        if ((ecopro['MACD'].iloc[i-1] < ecopro['Signal Line'].iloc[i-1]) and (ecopro['MACD'].iloc[i] > ecopro['Signal Line'].iloc[i])) or \
        ((ecopro['ema_50'].iloc[i-1] < ecopro['ema_200'].iloc[i-1]) and (ecopro['ema_50'].iloc[i] > ecopro['ema_200'].iloc[i])):
            signals.append(1)  # 매수 신호
        elif ((ecopro['MACD'].iloc[i-1] > ecopro['Signal Line'].iloc[i-1]) and (ecopro['MACD'].iloc[i] < ecopro['Signal Line'].iloc[i])) or \
        ((ecopro['ema_50'].iloc[i-1] > ecopro['ema_200'].iloc[i-1]) and (ecopro['ema_50'].iloc[i] < ecopro['ema_200'].iloc[i])):
            signals.append(0)  # 매도 신호
        else:
            signals.append(2)  # 교차 없음
ecopro['Signal'] = signals
ecopro1 = ecopro.iloc[1:, :]

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
# 랜덤포레스트 모델 학습 및 평가
# 특징 및 레이블 선택
X = ecopro1[['ema_50', 'ema_12', 'ema_26', 'ema_200', 'MACD', 'Signal Line']]
y = ecopro1['Signal']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# 랜덤포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# 특징 및 레이블 선택
X = ecopro1[['ema_50', 'ema_12', 'ema_26', 'ema_200', 'MACD', 'Signal Line']]
y = ecopro1['Signal']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 랜덤포레스트 모델 학습 (OOB 점수 활성화)
model = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

# OOB Score 확인
print(f"OOB Score: {model.oob_score_}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# 특징 및 레이블 선택
X = ecopro1[['ema_50', 'ema_12', 'ema_26', 'ema_200', 'MACD', 'Signal Line']]
y = ecopro1['Signal']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# SMOTE를 사용하여 오버샘플링
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 랜덤포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42)
model.fit(X_train_res, y_train_res)

# 예측 및 평가
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

# OOB Score 확인
print(f"OOB Score: {model.oob_score_}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 특징 및 레이블 선택
X = ecopro1[['ema_50', 'ema_12', 'ema_26', 'ema_200', 'MACD', 'Signal Line']]
y = ecopro1['Signal']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE를 사용하여 오버샘플링
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 150],  # 결정 트리의 개수
    'max_depth': [None, 10, 20, 30],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 내부 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],  # 리프 노드에 있어야 하는 최소 샘플 수
    'bootstrap': [True, False]  # 부트스트랩 샘플링 사용 여부
}

# RandomForestClassifier 인스턴스 생성
rf = RandomForestClassifier(random_state=42)

# GridSearchCV 인스턴스 생성
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# 모델 학습
grid_search.fit(X_train_res, y_train_res)

# 최적의 하이퍼파라미터 및 모델
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

best_model = grid_search.best_estimator_

# 테스트 데이터로 예측 및 평가
y_pred = best_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

# OOB Score 확인
print(f"OOB Score: {best_model.oob_score_ if hasattr(best_model, 'oob_score_') else 'OOB Score not available'}")


In [ ]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost as xgb

# KOSPI 데이터에서 주식 코드 추출
# KOSPI = fdr.StockListing('KOSPI')
# stock_codes = KOSPI['Code'].tolist()

# 사용하고 싶은 주식 코드의 수를 제한 
# stock_codes = stock_codes[:100]
# 분석할 주식 코드 목록
stock_codes = ['041190', '015750', '215200', '095660', '025980', '099320', '215000', '290650', '101160', '006730', '023160', '043150',
               '013030', '243070', '297890', '230360', '045100', '041830', '083450', '035890', '267980', '060250', '126700', '265520', 
               '018310', '050890', '035600', '051370', '148150', '054950', '211050', '299900', '078150', '118990', '108230', '950170', 
               '054450', '051500', '052400', '083310', '190510', '377450', '194700', '036890', '236200', '023760', '067280', '046440', 
               '123410', '136540', '214180', '092730', '036190', '023910', '452400', '298540', '091580', '079960', '009300', '023600', 
               '259630', '418470', '285490', '203650', '950190', '038110', '264450', '033290', '099440', '353810', '060560', '067900', 
               '092460', '100700', '382800', '066620', '066700', '013310', '124560', '332570', '021080', '137950', '069510', '215360',
               '365330', '066590', '058630', '053700', '005990', '071200', '011560', '035610', '089850', '036630', '352090', '142210', 
               '043260', '042500', '302430', '019590', '038460', '417790', '111710', '040420', '234300', '059210', '024880', '065710', 
               '086670', '054040', '012790', '263690', '014200', '009780', '052790', '122310', '241520', '036640', '216050', '208140', 
               '007370', '036670', '038680', '339950', '158430', '048430', '094970', '115440', '155650', '024800', '241790', '082210', 
               '130580', '054930', '224110', '212560', '090410', '250000', '105760', '271830', '053270', '094840', '010240', '038010', 
               '039420', '092300', '019540', '227950', '100030', '440290', '122690', '067920', '204020', '088910', '263920', '263770', 
               '098120', '003310', '263020', '019770', '332370', '246690', '111870', '066360', '046310', '025880', '078590', '131090', 
               '340440', '267790', '290270', '099410', '052460', '020180', 
               '297570', '362990', '006920', '042940', '080470', '039310', '113810', '154040', '208350', '043590', '335870', '053060']

# 빈 리스트를 만들어서 모든 주식 데이터를 통합할 준비
X = []
Y = []

# 각 주식 코드에 대해 데이터 불러오고 처리
for code in stock_codes:
    try:
        df = fdr.DataReader(code).dropna()  # 데이터 불러오기 및 결측값 제거
        
        # 주식 데이터가 비어있지 않은 경우에만 처리
        if not df.empty:
            close_prices = df['Close'].values

            # 지수 이동 평균 (EMA) 및 MACD 계산
            ema_12 = pd.Series(close_prices).ewm(span=12, adjust=False).mean().values
            ema_26 = pd.Series(close_prices).ewm(span=26, adjust=False).mean().values
            ema_50 = pd.Series(close_prices).ewm(span=50, adjust=False).mean().values
            ema_200 = pd.Series(close_prices).ewm(span=200, adjust=False).mean().values

            macd = ema_12 - ema_26
            signal_line = pd.Series(macd).ewm(span=9, adjust=False).mean().values

            # 윈도우 사이즈와 예측 기간 설정
            window_size = 10  # 윈도우 사이즈 (일수)
            next_date = 1     # 예측 기간 (일수)

            for i in range(len(close_prices) - window_size - next_date):
                window = close_prices[i:i + window_size]
                
                # EMA 및 MACD 지표 추가
                ema_window = ema_50[i:i + window_size]
                macd_window = macd[i:i + window_size]
                signal_window = signal_line[i:i + window_size]
                
                # 특성 벡터 생성
                features = np.concatenate([ema_window, macd_window, signal_window])
                
                # Signals 계산
                signals = [2]  # 초기값 (교차 없음)
                for j in range(1, len(macd)):
                    if ((macd[j-1] < signal_line[j-1]) and (macd[j] > signal_line[j])) or \
                       ((ema_50[j-1] < ema_200[j-1]) and (ema_50[j] > ema_200[j])):
                        signals.append(1)  # 매수 신호
                    elif ((macd[j-1] > signal_line[j-1]) and (macd[j] < signal_line[j])) or \
                         ((ema_50[j-1] > ema_200[j-1]) and (ema_50[j] < ema_200[j])):
                        signals.append(0)  # 매도 신호
                    else:
                        signals.append(2)  # 교차 없음
                
                # 타겟 값을 현재 윈도우의 마지막 인덱스에 대해 설정
                if i + window_size + next_date < len(signals):
                    target = signals[i + window_size + next_date]
                    X.append(features)
                    Y.append(target)
                
    except Exception as e:
        print(f"Error processing code {code}: {e}")

# X와 Y를 numpy 배열로 변환
X = np.array(X)
Y = np.array(Y)




In [9]:
# 데이터 shape 출력
print(f'X shape: {X.shape}')
print(f'Y shape: {Y.shape}')


X shape: (89645, 30)
Y shape: (89645,)


In [11]:
# 데이터 분할 (훈련 데이터와 테스트 데이터)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)

# XGBoost 분류 모델 생성 및 학습
model = xgb.XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss', 
    learning_rate=0.1, 
    n_estimators=100, 
    max_depth=5, 
    subsample=0.8, 
    colsample_bytree=0.8,
    scale_pos_weight=1,
    random_state=42
)
model.fit(X_train, Y_train)

# 테스트 데이터에 대해 예측 수행
Y_pred = model.predict(X_test)

# 모델 평가
accuracy = accuracy_score(Y_test, Y_pred)
print(f'Accuracy: {accuracy}')

conf_matrix = confusion_matrix(Y_test, Y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(Y_test, Y_pred)
print(f'Classification Report:\n{class_report}')

# 예측 결과 확인
print('Sample Predictions:')
for i in range(min(5, len(Y_pred))):
    print(f'Actual: {Y_test[i]}, Predicted: {Y_pred[i]}')


C:\Users\campus4D046\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:40:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9196274192648781
Confusion Matrix:
[[    0     0   722]
 [    0     0   718]
 [    1     0 16488]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       722
           1       0.00      0.00      0.00       718
           2       0.92      1.00      0.96     16489

    accuracy                           0.92     17929
   macro avg       0.31      0.33      0.32     17929
weighted avg       0.85      0.92      0.88     17929

Sample Predictions:
Actual: 2, Predicted: 2
Actual: 2, Predicted: 2
Actual: 2, Predicted: 2
Actual: 2, Predicted: 2
Actual: 2, Predicted: 2


C:\Users\campus4D046\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\campus4D046\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\campus4D046\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# 데이터 shape 출력
print(f'X shape: {X.shape}')
print(f'Y shape: {Y.shape}')

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

# XGBoost 모델 학습
model = xgb.XGBClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

X shape: (89645, 30)
Y shape: (89645,)
Accuracy: 0.9162808857158793
Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.00      0.00       722
           1       0.00      0.00      0.00       774
           2       0.92      1.00      0.96     16433

    accuracy                           0.92     17929
   macro avg       0.39      0.33      0.32     17929
weighted avg       0.85      0.92      0.88     17929



#### 예측했을 때 기업 식별자 코드도 같이 나오게

In [1]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost as xgb

# 분석할 주식 코드 목록 (기업 식별자 추가)
stock_codes = ['041190', '015750', '215200']

# 빈 리스트를 만들어서 모든 주식 데이터를 통합할 준비
X = []
Y = []
company_ids = []  # 기업 식별자를 저장할 리스트

# 각 주식 코드에 대해 데이터 불러오고 처리
for code in stock_codes:
    try:
        df = fdr.DataReader(code).dropna()  # 데이터 불러오기 및 결측값 제거
        
        # 주식 데이터가 비어있지 않은 경우에만 처리
        if not df.empty:
            close_prices = df['Close'].values

            # 지수 이동 평균 (EMA) 및 MACD 계산
            ema_12 = pd.Series(close_prices).ewm(span=12, adjust=False).mean().values
            ema_26 = pd.Series(close_prices).ewm(span=26, adjust=False).mean().values
            ema_50 = pd.Series(close_prices).ewm(span=50, adjust=False).mean().values
            ema_200 = pd.Series(close_prices).ewm(span=200, adjust=False).mean().values

            macd = ema_12 - ema_26
            signal_line = pd.Series(macd).ewm(span=9, adjust=False).mean().values

            # 기업 식별자를 원-핫 인코딩 형태로 생성
            company_id = np.zeros(len(stock_codes))
            company_id[stock_codes.index(code)] = 1

            # 윈도우 사이즈와 예측 기간 설정
            window_size = 10  # 윈도우 사이즈 (일수)
            next_date = 1     # 예측 기간 (일수)

            for i in range(len(close_prices) - window_size - next_date):
                window = close_prices[i:i + window_size]
                
                # EMA 및 MACD 지표 추가
                ema_window = ema_50[i:i + window_size]
                macd_window = macd[i:i + window_size]
                signal_window = signal_line[i:i + window_size]
                
                # 특성 벡터 생성 (기업 식별자 포함)
                features = np.concatenate([ema_window, macd_window, signal_window, company_id])
                
                # Signals 계산
                signals = [2]  # 초기값 (교차 없음)
                for j in range(1, len(macd)):
                    if ((macd[j-1] < signal_line[j-1]) and (macd[j] > signal_line[j])) or \
                       ((ema_50[j-1] < ema_200[j-1]) and (ema_50[j] > ema_200[j])):
                        signals.append(1)  # 매수 신호
                    elif ((macd[j-1] > signal_line[j-1]) and (macd[j] < signal_line[j])) or \
                         ((ema_50[j-1] > ema_200[j-1]) and (ema_50[j] < ema_200[j])):
                        signals.append(0)  # 매도 신호
                    else:
                        signals.append(2)  # 교차 없음
                
                # 타겟 값을 현재 윈도우의 마지막 인덱스에 대해 설정
                if i + window_size + next_date < len(signals):
                    target = signals[i + window_size + next_date]
                    X.append(features)
                    Y.append(target)
                    company_ids.append(stock_codes.index(code))  # 해당 데이터가 어떤 기업에 속하는지 기록
                
    except Exception as e:
        print(f"Error processing code {code}: {e}")

# X와 Y를 numpy 배열로 변환
X = np.array(X)
Y = np.array(Y)



In [ ]:
# 데이터 분할 (훈련 데이터와 테스트 데이터)
X_train, X_test, Y_train, Y_test, company_ids_train, company_ids_test = train_test_split(X, Y, company_ids, test_size=0.2, shuffle=True, random_state=42)

# XGBoost 분류 모델 생성 및 학습
model = xgb.XGBClassifier(
    objective='multi:softmax',  # 다중 클래스 분류를 위한 설정
    num_class=3,  # 세 가지 클래스 (0: 매도, 1: 매수, 2: 없음)
    eval_metric='mlogloss', 
    learning_rate=0.1, 
    n_estimators=100, 
    max_depth=5, 
    subsample=0.8, 
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, Y_train)

# 테스트 데이터에 대해 예측 수행
Y_pred = model.predict(X_test)

# 모델 평가
accuracy = accuracy_score(Y_test, Y_pred)
print(f'Accuracy: {accuracy}')

conf_matrix = confusion_matrix(Y_test, Y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(Y_test, Y_pred)
print(f'Classification Report:\n{class_report}')

# 예측 결과 확인 (기업 식별자 포함)
print('Sample Predictions:')
for i in range(min(5, len(Y_pred))):
    actual_company = stock_codes[company_ids_test[i]]
    print(f'Company: {actual_company}, Actual: {Y_test[i]}, Predicted: {Y_pred[i]}')
